## Table of Contents
* Load required packages & set working directory
* Function definitions
* Data organization
* Experiment 1 models
* Experiment 1 SHAP plots
* Experiment 2 models
* Experiment 2 SHAP plots

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from pytorch_tabnet.tab_model import TabNetClassifier
from random import sample
from itertools import chain
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

import os


from matplotlib import pyplot as plt
import random
#import netCDF4
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from collections import defaultdict
from tqdm import tqdm

## Set Working Directory #Make sure you're currently in the main repository folder.
#os.chdir("")

In [ ]:
#Functions for recording model results and parameters

def calculate_metrics_single(y_true, y_pred, duration, dataset, obs_col, model_name, model_run, smote,
                             test_set, features_size, label_imb_train, label_imb_test, 
                             train_size, test_size):
    res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run', 'smote', 'test-set',
     'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                                'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])
    # AUROC
    auc = metrics.roc_auc_score(y_true, y_pred)
    # AUPRC
    precision, recall, thresholds = metrics.precision_recall_curve(y_true, y_pred)
    avg_prec = metrics.auc(recall, precision)
    
    index_to_fill = res.shape[0]
    res.loc[index_to_fill, 'dataset'] = dataset
    res.loc[index_to_fill, 'obs_col'] = obs_col
    res.loc[index_to_fill, 'model_run'] = model_run
    #res.loc[index_to_fill, 'exp_type'] = exp_type
    res.loc[index_to_fill, 'smote'] = smote
    res.loc[index_to_fill, 'test-set'] = test_set
    res.loc[index_to_fill, 'precision'] = round(precision_score(y_true, y_pred), 2)
    res.loc[index_to_fill, 'accuracy'] = round(accuracy_score(y_true, y_pred), 2)
    res.loc[index_to_fill, 'recall'] = round(recall_score(y_true, y_pred), 2)
    res.loc[index_to_fill, 'F1'] = round(f1_score(y_true, y_pred), 2)
    res.loc[index_to_fill, 'auc'] = round(auc, 2)
    res.loc[index_to_fill, 'avg_prec'] = round(avg_prec, 2)
    res.loc[index_to_fill, 'model'] = model_name
    res.loc[index_to_fill, 'feat.'] = features_size
    res.loc[index_to_fill, 'label-imb-train'] = round(label_imb_train, 2).item()
    res.loc[index_to_fill, 'label-imb-test'] = round(label_imb_test, 2).item()
    res.loc[index_to_fill, 'train-size'] = train_size
    res.loc[index_to_fill, 'test-size'] = test_size
    res.loc[index_to_fill, 'runtime(sec)'] = round(duration, 2)
    return res

#To test binning; not used in final results
def calculate_metrics_multiclass(y_true, y_pred, duration, dataset, obs_col, model_name, model_run, smote,
                             test_set, features_size, label_imb_train, label_imb_test, 
                             train_size, test_size):
    res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run', 'smote',
     'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                                'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])
    index_to_fill = res.shape[0]
    #auc = metrics.roc_auc_score(y_true, y_pred)
    # AUPRC
    #precision, recall, thresholds = metrics.precision_recall_curve(y_true, y_pred)
    #avg_prec = metrics.auc(recall, precision)
    res.loc[index_to_fill, 'dataset']= dataset
    res.loc[index_to_fill, 'obs_col']=obs_col
    res.loc[index_to_fill, 'model_run']=model_run
    #res.loc[index_to_fill, 'exp_type'] = exp_type
    res.loc[index_to_fill, 'smote'] = smote
    res.loc[index_to_fill, 'test-set'] = test_set
    res.loc[index_to_fill, 'precision'] = round(precision_score(y_true, y_pred, average='micro'), 2)
    res.loc[index_to_fill, 'accuracy'] = round(accuracy_score(y_true, y_pred), 2)
    res.loc[index_to_fill, 'recall'] = round(recall_score(y_true, y_pred, average='micro'), 2)
    res.loc[index_to_fill, 'F1'] = round(f1_score(y_true, y_pred, average='micro'), 2)
    #res.loc[index_to_fill, 'auc'] = round(auc, 2)
    #res.loc[index_to_fill, 'avg_prec'] = round(avg_prec, 2)
    res.loc[index_to_fill, 'model'] = model_name
    res.loc[index_to_fill, 'feat.'] = features_size
    res.loc[index_to_fill, 'label-imb-train'] = ' '.join(str(x) for x in round(label_imb_train,2))
    res.loc[index_to_fill, 'label-imb-test'] = ' '.join(str(x) for x in round(label_imb_test,2))
    res.loc[index_to_fill, 'train-size'] = train_size
    res.loc[index_to_fill, 'test-size'] = test_size
    res.loc[index_to_fill, 'runtime(sec)'] = round(duration, 2)
    return res

def interpret_ml(x_train, x_test, x_header_names, model, output_directory, case):
    import shap
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x_test)

    shap_values_idx = 1

    # save shap values in tabular format
    if not os.path.exists(output_directory):
        os.mkdir(output_directory)
    shap_values_df = pd.DataFrame(data=shap_values[shap_values_idx], columns=[i + '_shp' for i in x_header_names])
    shap_values_df.to_csv(os.path.join(output_directory, 'shap-values-RF-' + case + '.csv'), index=False)

    plt.clf()
    shap.summary_plot(shap_values[shap_values_idx], x_test, feature_names=x_header_names, show=False)
    plt.yticks(fontname="Times New Roman")
    plt.xticks(fontname="Times New Roman")
    plt.tick_params(axis='both', which='major', labelsize=23)
    plt.xlabel('SHAP value (impact on model output - pos. class)', fontname="Times New Roman", fontsize=22)
    plt.savefig(os.path.join(output_directory, 'shap-summary-plot-RF-' + case + '.eps'), bbox_inches='tight', format='eps') #For print quality figures
    plt.savefig(os.path.join(output_directory, 'shap-summary-plot-RF-' + case + '.png'), bbox_inches='tight')
    pd.DataFrame(x_test).to_csv(os.path.join(output_directory, case + '-test.csv')) #For mapping results
    return shap_values

In [ ]:
# Helper functions
def get_cols(data_coll, allVars):
    resp_cols = []
    for var in allVars:
        varcols=[col for col in data_coll.columns if col.startswith(var)]
        resp_cols.append(varcols)
    resp_cols=list(chain(*resp_cols))
    column_indices = [data_coll.columns.get_loc(col) for col in resp_cols]
    return(resp_cols, column_indices)

def get_best_vars(varlabels, resp_cols):
    colnames=[]
    for i in varlabels:
        index=i.replace("x","")
        colnames.append(resp_cols[int(index)])
    return(colnames)

def save_preds(data, y_true, y_pred, outpath):
    data_out = data[["enum", "month", "obs_year", "lat", "lon", "rural", "cat"]]
    data_out.loc[:,"obs"]=y_true
    data_out.loc[:,"pred"]=y_pred
    data_out['pred_type']=np.where((data_out['obs']==0) & (data_out['pred']==0), 'tn', 
                                    np.where((data_out['obs']==1) & (data_out['pred']==0), 'fn',
                                             np.where((data_out['obs']==0) & (data_out['pred']==1), 'fp', 'tp')))
    data_out.to_csv(outpath)

In [ ]:
#Compiling the single dataset - this can be skipped
hunger_lc=pd.read_csv("Datasets/hunger_lc.csv", index_col=0)
mzprice = pd.read_csv("Datasets/mzprice_final.csv")
sat_final = pd.read_csv("Datasets/sat_final.csv", index_col=0)
ea_regions = pd.read_csv("Datasets/ea_regions.csv")
ea_dists = pd.read_csv("ea_dists.csv")
lsms_vars = pd.read_csv("Datasets/ea_geovars_all.csv")
panel=pd.read_csv("Datasets/panel_eas.csv")
mzprice_infl = pd.read_csv("Datasets/mzprice_final_annualchg.csv", index_col=0)
regionnums = pd.DataFrame({"region" : ["North", "Central", "South"], "regionnum": [1,2,3]})
hunger_lc=hunger_lc.drop_duplicates().reset_index(drop=True) #Forgot to do this step on export.

print(len(sat_final))
data_coll = sat_final.merge(lsms_vars, on=['ea_id', 'wave'])
print(len(data_coll))
data_coll = data_coll.merge(panel, on=['ea_id'], how='inner')
print(len(data_coll))
data_coll = data_coll.merge(hunger_lc, on=['ea_id','obs_year'])
print(len(data_coll))
data_coll = data_coll.merge(mzprice, on=['ea_id', 'obs_year','obs_monthnum'])
print(len(data_coll))
data_coll = data_coll.merge(ea_regions, on="ea_id")
print(len(data_coll))
data_coll = data_coll.merge(regionnums, on="region")
print(len(data_coll))
data_coll=data_coll.merge(mzprice_infl, on=['ea_id', 'obs_year','obs_monthnum'])
print(len(data_coll))
data_coll = data_coll.merge(ea_dists[["ea_id","distnum"]], on="ea_id")
data_coll=data_coll.rename(columns={"ea_id" : "enum"})
data_coll=data_coll.loc[:,~data_coll.columns.str.startswith(('year_offset', 'obs_month_', 'nightlights'))] #Cruft; nightlights too messy to use directly.
data_coll["cat_cat1"]=np.where(data_coll["cat"]>2, 1,0)
data_coll["cat_cat2"]=np.where(data_coll["cat"]>3, 1,0)
region_dummies = pd.get_dummies(data_coll['region'])
data_coll=data_coll.drop(['region', 'regionnum', 'lsms_aez'], axis=1)
data_coll=data_coll.join(region_dummies)
data_coll['offset']=data_coll['obs_monthnum']-3
data_coll['offset']=np.where(data_coll['offset']<0, data_coll['offset']+12, data_coll['offset'])
data_coll=data_coll.rename(columns={"South": "region_south", "North": "region_north", "Central": "region_central"})
data_coll['qtr1']=np.where((data_coll['obs_monthnum']==12) | (data_coll['obs_monthnum']==1) | (data_coll['obs_monthnum']==2),
                          1, 0)
data_coll['qtr2']=np.where((data_coll['obs_monthnum']==3) | (data_coll['obs_monthnum']==4) | (data_coll['obs_monthnum']==5),
                          1,0)
data_coll['qtr3']=np.where((data_coll['obs_monthnum']==6) | (data_coll['obs_monthnum']==7) | (data_coll['obs_monthnum']==8),
                           1, 0)
data_coll['mzprice_qtrchg']=(data_coll['mzprice_lag1']+data_coll['mzprice_lag2']+data_coll['mzprice_lag3'])/(data_coll['mzprice_lag4']+data_coll['mzprice_lag5']+data_coll['mzprice_lag6'])

In [ ]:
genl_infl = pd.read_csv("Datasets/monthly_inflation.csv")
for i in range(1,13):
    genl_infl['obs_monthnum'] = genl_infl['obs_monthnum']+1
    genl_infl['obs_year'] = np.where(genl_infl['obs_monthnum'] > 12, genl_infl['obs_year']+1, genl_infl['obs_year'])
    genl_infl['obs_monthnum']=np.where(genl_infl['obs_monthnum']>12, genl_infl['obs_monthnum']-12, genl_infl['obs_monthnum'])
    genl_infl_merge=genl_infl.copy()
    genl_infl_merge[f'inflationcpi_lag{i}']=genl_infl_merge['inflation']
    genl_infl_merge=genl_infl_merge.drop(columns=['inflation'])
    data_coll = data_coll.merge(genl_infl_merge, on=['obs_year', 'obs_monthnum'])
    
mzmeans = pd.read_csv("Datasets/mznorms.csv", index_col=0)
mzmeans = mzmeans.rename(columns={"obs_month": "obs_monthnum"})
for i in range(1,13):
    mzmeans['obs_monthnum'] = mzmeans['obs_monthnum']+1
    mzmeans['obs_year'] = np.where(mzmeans['obs_monthnum'] > 12, mzmeans['obs_year']+1, mzmeans['obs_year'])
    mzmeans['obs_monthnum']=np.where(mzmeans['obs_monthnum']>12, mzmeans['obs_monthnum']-12, mzmeans['obs_monthnum'])
    mzmeans_merge=mzmeans.copy()
    mzmeans_merge[f'mz_mean_lag{i}']=mzmeans_merge['mz_mean']
    mzmeans_merge[f'mz_sd_lag{i}']=mzmeans_merge['mz_sd']
    mzmeans_merge=mzmeans_merge.drop(columns=['mz_mean', 'mz_sd'])
    data_coll = data_coll.merge(mzmeans_merge, on=['obs_year', 'obs_monthnum'])
    data_coll[f'mznorm_lag{i}']=(data_coll[f'mzprice_lag{i}']-data_coll[f'mz_mean_lag{i}'])/data_coll[f'mz_sd_lag{i}']

In [ ]:
data_coll.to_csv("data_coll_allvars.csv", index=False)

In [ ]:
#Using normalized climatological variables instead; not in final paper.
data_coll_norms = data_coll.copy()
norms = pd.read_csv("Datasets/TerraClimateNorms.csv")
norms = norms.loc[:, ~norms.columns.str.startswith('pdsi')] #Already normalized.
data_coll_norms = data_coll_norms.merge(norms, left_on='enum',  right_on="ea_id")
print(len(data_coll_norms))
tc_vars = ['ppt','soil','vap','vpd']
for var in tc_vars:
    for month in months:
        data_coll_norms[f'norm_{var}_{month}'] = (data_coll_norms[f'{var}_{month}']-data_coll_norms[f'{var}_mean_{month}'])/data_coll_norms[f'{var}_sd_{month}']


In [ ]:
#Condensing some variables to the season level; not in final paper
data_coll_season = data_coll.copy()
data_coll_season = data_coll_season.loc[:, ~data_coll_season.columns.str.startswith(('ppt', 'ndvi', 'lc'))]
data_coll_season = data_coll_season.loc[:, ~data_coll_season.columns.str.endswith(('may','jun','jul','aug','sep'))]
data_coll_season['ppt_wet']=data_coll['ppt_oct']+data_coll['ppt_dec']+data_coll['ppt_jan']+data_coll['ppt_feb']+data_coll['ppt_mar']+data_coll['ppt_apr']
data_coll_season['ppt_max']=data_coll[['ppt_oct', 'ppt_dec', 'ppt_jan', 'ppt_feb', 'ppt_mar', 'ppt_apr']].max(axis=1)
gr_seas=['sep','oct','nov','dec','jan','feb','mar','apr']
for mon in range(1,8):
    data_coll_season[f'ndvi_chg_{gr_seas[mon]}']=data_coll[f'ndvi_{gr_seas[mon]}']-data_coll[f'ndvi_{gr_seas[(mon-1)]}']

### Experiment 1

In [ ]:
obs_cols=["cat_cat1", "cat_cat2"]
#Leftover
test_fraction=0.2
model_names = ['logistic-regression', 'lasso', 'random-forest','tabnet']
#model_names=['random-forest']
naive_vars = ['region', 'qtr', 'rural']
satVars = ['ndvi', 'pdsi', 'ppt', 'soil', 'vap', 'vpd', 
               'tmin', 'tmax', 'tmean', 'lcurb', 'lcforst', 
               'lcgrass','lccrop'] 
priceVars = ['mzprice']
priceNorm = ['mznorm']
surveys = [1,3,4]
lsmsVars = ['lsms']
inflVars = ['mzinfl', 'inflation']
dataset='normal_allvars'

model_runs = ["naive","remote","price",'lsms', 'infl', 'infl+lsms',
              'infl+remote', "price+infl", "price+remote","price+lsms", 
              "sat+lsms", "selected", 'infl+remote-naive', "all"]

for obs_col in obs_cols:
    res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run',  'smote', 'test-set',
 'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                            'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])
    outdir=f"Experiment 1 - {obs_col[-4:]}"
    selected_features = pd.DataFrame(columns=["wave", "dataset_num", "obs_col", "selectedvars"])
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    if not os.path.exists(os.path.join(outdir, "Confusion Matrices")):
        os.mkdir(os.path.join(outdir, "Confusion Matrices"))

    if obs_col=="cat_cat1":
        smoteout=False
    else: 
        smoteout=True
    for ds in tqdm(range(1,6)):
        for survey in tqdm(surveys):
            data_coll_wave=pd.read_csv(f"Subsamples/data_coll_samp{ds}_wave{survey}_train.csv")
            #Unused columns
            data_coll_wave.drop(columns=['mzprice_yoy', 'mzprice_qtrchg', 'region_central'], inplace=True)
            data_coll=pd.read_csv(f'Subsamples/data_coll_samp{ds}_wave{survey}_test.csv')
            data_coll.drop(columns=['mzprice_yoy', 'mzprice_qtrchg','region_central'],inplace=True)
    
            clf = Pipeline([
                ('scale', StandardScaler()),
                ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=1000, random_state=0)))
            ])
            allVars=naive_vars + satVars + priceVars+lsmsVars+inflVars
            resp_cols, column_indices = get_cols(data_coll, allVars)
            X_train = data_coll_wave.loc[:, resp_cols]
            y_train = data_coll_wave.loc[:, obs_col] 
    
            clf.fit(X_train, y_train.values.ravel())
            selecVars=clf[:-1].get_feature_names_out()
            for model_name in tqdm(model_names):
                model=None
                if model_name == 'logistic-regression':
                    model = LogisticRegression(penalty=None)
                if model_name == 'lasso':
                    model= LogisticRegression(penalty='l1', solver='liblinear')
                if model_name == 'random-forest':
                    model = RandomForestClassifier(random_state=42)
                if model_name == 'tabnet':
                    model = TabNetClassifier(verbose=0,seed=42)
    
                for model_run in tqdm(model_runs):
                    if model_run=='naive':
                        allVars=naive_vars
                    elif model_run=="remote":
                        allVars= satVars+naive_vars
                    elif model_run=="price":
                        allVars= priceVars+naive_vars
                    elif model_run=="price+infl":
                        allVars=priceVars+inflVars+naive_vars
                    elif model_run=="price+lsms":
                        allVars=priceVars+lsmsVars+naive_vars
                    elif model_run=="lsms": 
                        allVars=lsmsVars+naive_vars
                    elif model_run=="sat+lsms":
                        allVars=satVars+lsmsVars+naive_vars
                    elif model_run=="infl+lsms":
                        allVars=inflVars+lsmsVars
                    elif model_run=="infl+remote":
                        allVars=inflVars+naive_vars+satVars
                    elif model_run=="infl+remote-naive":
                        allVars=inflVars+satVars
                    elif model_run=="selected":
                        allVars=selecVars
                    elif model_run=="infl":
                        allVars=inflVars
                    elif model_run=="all":
                        allVars=naive_vars + satVars + priceVars + lsmsVars+inflVars
                        
                    if(model_run=="selected"):
                        resp_cols=allVars
                    else:
                        resp_cols, _ = get_cols(X_train, allVars)
    
                    X_train_g=X_train.loc[:, resp_cols]
                    y_train_g=y_train
                    if (smoteout==True) & (model_run!="naive"):
                        cat_vars = []
                        if 'region' in allVars:
                            cat_vars = cat_vars + ['region_south','region_north']
                        if 'rural' in allVars: 
                            cat_vars = cat_vars + ['rural']
                        if 'qtr' in allVars:
                            cat_vars = cat_vars + ['qtr1','qtr2','qtr3']
                        if len(cat_vars) > 0:     
                            smote=SMOTENC(categorical_features=cat_vars, random_state=42)
                        else:
                            smote=SMOTE(random_state=42)
                        X_train_g, y_train_g=smote.fit_resample(X_train_g, y_train_g)
                        smote=True
                    else:
                        smote=False
    
                    ts_df=data_coll.loc[:, :].reset_index(drop=True)
    
                    #resp_cols, _ = get_cols(data_coll, allVars)
                    X_test_g=ts_df.loc[:, resp_cols]
                    y_test_g=ts_df.loc[:, obs_col]
    
                    scaler = StandardScaler()
                    X_train_g = scaler.fit_transform(X_train_g)
                    X_test_g = scaler.transform(X_test_g)
                    start_time = time.time()
                    model.fit(X_train_g, y_train_g.values.ravel())
                    duration = time.time() - start_time
                    y_pred = model.predict(X_test_g)
    
                    # create confusion matrix
                    cm = confusion_matrix(y_test_g, y_pred)
                    cm=pd.DataFrame(cm)
                    cm=cm.rename(index={0:"True0", 1:"True1"}, columns={0: "Pred0", 1:"Pred1"})
    
    
                    test_set=f'{survey}'
    
    
                    features_size = X_train_g.shape[1]
                    label_imb_train = y_train_g.sum()/len(y_train_g)
                    label_imb_test = y_test_g.sum()/len(y_test_g)
                    #using imbalance ratio per label for non-binary data
                    train_size = X_train_g.shape[0]
                    test_size = X_test_g.shape[0]
                    res= pd.concat([res, calculate_metrics_single(y_test_g, y_pred, duration, dataset, obs_col, model_name, model_run, 
                                                        smote, test_set, features_size, label_imb_train, label_imb_test, 
                                                        train_size, test_size)], axis=0)
    
    
                    if not os.path.exists(os.path.join(outdir, "Preds")):
                        os.mkdir(os.path.join(outdir, "Preds"))
                    if not os.path.exists(os.path.join(outdir, f"Preds/{model_name}")):
                        os.mkdir(os.path.join(outdir, f"Preds/{model_name}"))
                    save_preds(ts_df, y_test_g, y_pred, os.path.join(outdir, f"Preds/{model_name}", f'preds_{obs_col}_{model_run}_r{ds}_wave{survey}.csv'))
    
                    if not os.path.exists(os.path.join(outdir, "Confusion Matrices")):
                        os.mkdir(os.path.join(outdir, "Confusion Matrices"))
                    if not os.path.exists(os.path.join(outdir, f"Confusion Matrices/{model_name}")):
                        os.mkdir(os.path.join(outdir, f"Confusion Matrices/{model_name}"))
                    cm.to_csv(os.path.join(outdir, f'Confusion Matrices/{model_name}/{obs_col}_{model_run}_r{ds}_wave{survey}.csv'))
                    if(model_run=="selected"):
                        index=len(selected_features)
                        selected_features.loc[index, "wave"]=survey
                        selected_features.loc[index, "dataset_num"]=ds
                        selected_features.loc[index,"selected_features"]=" ".join(allVars)
                        selected_features.loc[index, "obs_col"]=obs_col
                    #shap_output_dir=os.path.join(outdir, f'SHAP_{obs_col}_{model_name}_{model_run}_w{wave}')
                    #modelout = model.steps[-1][1]
                    #interpret_ml(X_train_g, X_test_g, resp_cols, modelout, shap_output_dir, model_run)
                
    res.to_csv(os.path.join(outdir, f'Exp1 Results {obs_col[-4:]}.csv'))
    selected_features.to_csv(os.path.join(outdir, f'Exp1 Selected Features {obs_col[-4:]}.csv'))

## SHAP Plots

In [ ]:
obs_cols=["cat_cat1", "cat_cat2"]
#model_names = ['logistic-regression', 'ridge', 'random-forest','tabnet']
model_names=['random-forest']
naive_vars = ['region', 'qtr', 'rural']
satVars = ['ndvi', 'pdsi', 'ppt', 'soil', 'vap', 'vpd', 
               'tmin', 'tmax', 'tmean', 'lcurb', 'lcforst', 
               'lcgrass','lccrop'] 
priceVars = ['mzprice']
priceNorm = ['mznorm']
surveys = [1,3,4]
lsmsVars = ['lsms']
inflVars = ['mzinfl', 'inflation']
dataset='normal_allvars'
#model_runs = ["naive","remote","price",'lsms', 'infl', 'pricenorm',
#              'pricenorm+infl', 'pricenorm+remote', 'pricenorm+lsms',
#              "price+infl", "price+remote","price+lsms", 
#              "sat+lsms", "selected", "all"]
model_runs=['all']

for obs_col in obs_cols:
    outdir=f"Experiment 1 - SHAP"
    res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run',  'smote', 'test-set',
     'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                                'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])
    
    
    
    selected_features = pd.DataFrame(columns=["wave", "dataset_num", "obs_col", "selectedvars"])
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    if obs_col=="cat_cat1":
        smoteout=False
    else: 
        smoteout=True
    for survey in surveys:
        for ds in range(3,4): #Just take the third dataset to reduce processing time - first one did not produce good fits as discussed in paper; 2-5 can be added to view similarity across samples
            data_coll_wave=pd.read_csv(f"Subsamples/data_coll_samp{ds}_wave{survey}_train.csv")
                #Unused columns
            data_coll_wave.drop(columns=['mzprice_yoy', 'mzprice_qtrchg', 'region_central'], inplace=True)
            data_coll_test=pd.read_csv(f'Subsamples/data_coll_samp{ds}_wave{survey}_test.csv')
            data_coll_test.drop(columns=['mzprice_yoy', 'mzprice_qtrchg','region_central'],inplace=True)
            
            for model_name in model_names:
                model = None
                if model_name == 'logistic-regression':
                    model = LogisticRegression(penalty='none')
                if model_name == 'lasso':
                    model= LogisticRegression(penalty='l1', solver='liblinear')
                if model_name == 'random-forest':
                    model = RandomForestClassifier()
                if model_name == 'tabnet':
                    model = TabNetClassifier(verbose=0,seed=42)
    
                for obs_col in tqdm(obs_cols):
                    for model_run in tqdm(model_runs):
                        if model_run=='naive':
                            allVars=naive_vars
                        elif model_run=="remote":
                            allVars= satVars+naive_vars
                        elif model_run=="price":
                            allVars= priceVars+naive_vars
                        elif model_run=="price+infl":
                            allVars=priceVars+inflVars+naive_vars
                        elif model_run=="price+lsms":
                            allVars=priceVars+lsmsVars+naive_vars
                        elif model_run=="lsms": 
                            allVars=lsmsVars+naive_vars
                        elif model_run=="sat+lsms":
                            allVars=satVars+lsmsVars+naive_vars
                        elif model_run=="infl+lsms":
                            allVars=inflVars+lsmsVars
                        elif model_run=="infl+remote":
                            allVars=inflVars+naive_vars+satVars
                        elif model_run=="infl+remote-naive":
                            allVars=inflVars+satVars
                        elif model_run=="selected":
                            allVars=selecVars
                        elif model_run=="infl":
                            allVars=inflVars+naive_vars
                        elif model_run=="all":
                            allVars=naive_vars + satVars + priceVars + lsmsVars+inflVars
    
                        if model_run!="selected":
                            resp_cols, _ = get_cols(data_coll_wave, allVars)
                        else:
                            resp_colls="allVars"
    
                        X_train_g=data_coll_wave.loc[:, resp_cols]
                        y_train_g=data_coll_wave.loc[:, obs_col]
                        ts_df=data_coll_test
                        #ts_df=data_coll_test.loc[data_coll['wave']==survey, :].reset_index(drop=True)
                        X_test_g=ts_df.loc[:, resp_cols]
                 
                        
                        if (smoteout==True) & (model_run!="naive"):
                            cat_vars = []
                            if 'region' in allVars:
                                cat_vars = cat_vars + ['region_south','region_north']
                            if 'rural' in allVars:
                                cat_vars = cat_vars + ['rural']
                            if 'qtr' in allVars:
                                cat_vars = cat_vars + ['qtr1','qtr2','qtr3']
                            if len(cat_vars) > 0:     
                                smote=SMOTENC(categorical_features=cat_vars, random_state=42)
                            else:
                                smote=SMOTE(random_state=42)
                            X_train_g, y_train_g=smote.fit_resample(X_train_g, y_train_g)
                            smote=True
                        else:
                            smote=False
                        
                        
                        scaler = StandardScaler()
                        X_train_g = scaler.fit_transform(X_train_g)
                        X_test_g = scaler.transform(X_test_g)
                        start_time = time.time()
                        model.fit(X_train_g, y_train_g.values.ravel())
                        duration = time.time() - start_time
                        #y_pred = model.predict(X_test_g)
                        res= pd.concat([res, calculate_metrics_single(y_test_g, y_pred, duration, dataset, obs_col, model_name, model_run, 
                                                        smote, test_set, features_size, label_imb_train, label_imb_test, 
                                                        train_size, test_size)], axis=0)
                        shap_output_dir=os.path.join(outdir, f'SHAP_{obs_col}_{model_name}_{model_run}_w{survey}')
                        #modelout = model.steps[-1][1]
                        interpret_ml(X_train_g, X_test_g, resp_cols, model, shap_output_dir, model_run)
            
    res.to_csv(os.path.join(outdir, f'Exp1 Results {obs_col}.csv'))

In [ ]:
data_coll_test

### Experiment 2

In [ ]:
obs_cols=["cat_cat1", "cat_cat2"]
model_names = ['logistic-regression', 'lasso', 'random-forest','tabnet']
naive_vars = ['region', 'qtr', 'rural']
satVars = ['ndvi', 'pdsi', 'ppt', 'soil', 'vap', 'vpd', 
               'tmin', 'tmax', 'tmean', 'lcurb', 'lcforst', 
               'lcgrass','lccrop'] 
priceVars = ['mzprice']
priceNorm = ['mznorm']
surveys = [2,3,4]
lsmsVars = ['lsms']
inflVars = ['mzinfl', 'inflation']
dataset='normal_allvars'
#smotes = [True, False]
smoteout=True #Can be modified to compare smote/smoteless sampling.       
#if not already loaded in
#data_coll=pd.read_csv("data_coll_allvars.csv")
#data_coll=data_coll.drop(columns=['mzprice_yoy', 'region_central', 'mzprice_qtrchg'])

model_runs = ["naive","remote","price",'lsms', 'infl', 'infl+lsms',
              'infl+remote', "price+infl", "price+remote","price+lsms", 
              "sat+lsms", "selected", 'infl+remote-naive', "all"]

for obs_col in obs_cols:
    outdir=f"Experiment 2 - {obs_col}"
    res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run',  'smote', 'test-set',
 'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                            'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])
    selected_features = pd.DataFrame(columns=["wave", "dataset_num", "obs_col", "selectedvars"])
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    if not os.path.exists(os.path.join(outdir, "Confusion Matrices")):
        os.mkdir(os.path.join(outdir, "Confusion Matrices"))

    if obs_col=="cat_cat1":
        smoteout=False
    else: 
        smoteout=True
    
    for ds in range(1,6):
        data_coll_wave=pd.read_csv(f"Subsamples/data_coll_samp{ds}_train_cat1.csv") #This has both cat1 and cat2
        data_coll_wave=data_coll_wave.drop(columns=['mzprice_yoy', 'region_central', 'mzprice_qtrchg'])
        for survey in surveys:
            clf = Pipeline([
                ('scale', StandardScaler()),
                ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=1000, random_state=0)))
            ])
            allVars=naive_vars + satVars + priceVars+lsmsVars+inflVars
            resp_cols, column_indices = get_cols(data_coll, allVars)
            X_train = data_coll_wave.loc[data_coll_wave['wave']<survey, resp_cols]
            y_train = data_coll_wave.loc[data_coll_wave['wave']<survey, obs_col] 
    
            clf.fit(X_train, y_train.values.ravel())
            selecVars=clf[:-1].get_feature_names_out()
            
            for model_name in model_names:
                model = None
                if model_name == 'logistic-regression':
                    model = LogisticRegression(penalty=None)
                if model_name == 'lasso':
                    model= LogisticRegression(penalty='l1', solver='liblinear')
                if model_name == 'random-forest':
                    model = RandomForestClassifier()
                if model_name == 'tabnet':
                    model = TabNetClassifier(verbose=0,seed=42)
    
                for obs_col in tqdm(obs_cols):
                    for model_run in tqdm(model_runs):
                        if model_run=='naive':
                            allVars=naive_vars
                        elif model_run=="remote":
                            allVars= satVars+naive_vars
                        elif model_run=="price":
                            allVars= priceVars+naive_vars
                        elif model_run=="price+infl":
                            allVars=priceVars+inflVars+naive_vars
                        elif model_run=="price+lsms":
                            allVars=priceVars+lsmsVars+naive_vars
                        elif model_run=="lsms": 
                            allVars=lsmsVars+naive_vars
                        elif model_run=="sat+lsms":
                            allVars=satVars+lsmsVars+naive_vars
                        elif model_run=="infl+lsms":
                            allVars=inflVars+lsmsVars
                        elif model_run=="infl+remote":
                            allVars=inflVars+naive_vars+satVars
                        elif model_run=="infl+remote-naive":
                            allVars=inflVars+satVars
                        elif model_run=="selected":
                            allVars=selecVars
                        elif model_run=="infl":
                            allVars=inflVars+naive_vars
                        elif model_run=="all":
                            allVars=naive_vars + satVars + priceVars + lsmsVars+inflVars
    
                        if model_run!="selected":
                            resp_cols, _ = get_cols(X_train, allVars)
                        else:
                            resp_colls="allVars"
    
                        X_train_g=X_train.loc[:, resp_cols]
                        y_train_g=y_train
                        
                        ts_df=data_coll.loc[data_coll['wave']==survey, :].reset_index(drop=True)
                        #resp_cols, _ = get_cols(data_coll, allVars)
                        X_test_g=ts_df.loc[:, resp_cols]
                        y_test_g=ts_df.loc[:, obs_col]

                        
                        if (smoteout==True) & (model_run!="naive"):
                            cat_vars = []
                            if 'region' in allVars:
                                cat_vars = cat_vars + ['region_south','region_north']
                            if 'rural' in allVars:
                                cat_vars = cat_vars + ['rural']
                            if 'qtr' in allVars:
                                cat_vars = cat_vars + ['qtr1','qtr2','qtr3']
                            if len(cat_vars) > 0:     
                                smote=SMOTENC(categorical_features=cat_vars, random_state=42)
                            else:
                                smote=SMOTE(random_state=42)
                            X_train_g, y_train_g=smote.fit_resample(X_train_g, y_train_g)
                            smote=True
                        else:
                            smote=False
                        
                        
                        scaler = StandardScaler()
                        X_train_g = scaler.fit_transform(X_train_g)
                        X_test_g = scaler.transform(X_test_g)
                        start_time = time.time()
                        model.fit(X_train_g, y_train_g.values.ravel())
                        duration = time.time() - start_time
                        y_pred = model.predict(X_test_g)
    
                        # create confusion matrix
                        cm = confusion_matrix(y_test_g, y_pred)
                        cm=pd.DataFrame(cm)
                        cm.rename(index={0:"True0", 1:"True1"}, columns={0: "Pred0", 1:"Pred1"})
    
                        test_set=f'round{ds}_survey{survey}'
    
    
                        features_size = X_train_g.shape[1]
                        label_imb_train = y_train_g.sum()/len(y_train_g)
                        label_imb_test = y_test_g.sum()/len(y_test_g)
                        #using imbalance ratio per label for non-binary data
    
    
                        train_size = X_train_g.shape[0]
                        test_size = X_test_g.shape[0]
    
                        res= pd.concat([res, calculate_metrics_single(y_test_g, y_pred, duration, dataset, obs_col, model_name, model_run, 
                                                            smote, test_set, features_size, label_imb_train, label_imb_test, 
                                                            train_size, test_size)], axis=0)
    
    
                        if not os.path.exists(os.path.join(outdir, "Preds")):
                            os.mkdir(os.path.join(outdir, "Preds"))
                        if not os.path.exists(os.path.join(outdir, f"Preds/{model_name}")):
                            os.mkdir(os.path.join(outdir, f"Preds/{model_name}"))
                        save_preds(ts_df, y_test_g, y_pred, os.path.join(outdir, f"Preds/{model_name}", f'preds_{obs_col}_{model_run}_r{ds}_wave{survey}.csv'))
    
                        if not os.path.exists(os.path.join(outdir, "Confusion Matrices")):
                            os.mkdir(os.path.join(outdir, "Confusion Matrices"))
                        if not os.path.exists(os.path.join(outdir, f"Confusion Matrices/{model_name}")):
                            os.mkdir(os.path.join(outdir, f"Confusion Matrices/{model_name}"))
                        cm.to_csv(os.path.join(outdir, f'Confusion Matrices/{model_name}/{obs_col}_{model_run}_r{ds}_wave{survey}.csv'))
                        if(model_run=="selected"):
                            index=len(selected_features)
                            selected_features.loc[index, "wave"]=survey
                            selected_features.loc[index, "dataset_num"]=ds
                            selected_features.loc[index,"selected_features"]=" ".join(allVars)
                            selected_features.loc[index, "obs_col"]=obs_col
                        #shap_output_dir=os.path.join(outdir, f'SHAP_{obs_col}_{model_name}_{model_run}_w{survey}')
                        #modelout = model.steps[-1][1]
                        #To do SHAP without a separate code block:
                        #interpret_ml(X_train_g, X_test_g, resp_cols, model, shap_output_dir, model_run)
                
    res.to_csv(os.path.join(outdir, f"Exp2 {obs_col} Results.csv"))
    selected_features.to_csv(os.path.join(outdir, f"Exp2 {obs_col} Selected Features.csv"))

### SHAP Plots

In [ ]:
obs_cols=["cat_cat1", "cat_cat2"]
model_names=['random-forest']
naive_vars = ['region', 'qtr', 'rural']
satVars = ['ndvi', 'pdsi', 'ppt', 'soil', 'vap', 'vpd', 
               'tmin', 'tmax', 'tmean', 'lcurb', 'lcforst', 
               'lcgrass','lccrop'] 
priceVars = ['mzprice']
priceNorm = ['mznorm']
surveys = [2,3,4]

lsmsVars = ['lsms']
inflVars = ['mzinfl', 'inflation']

dataset='normal_allvars'
#smotes = [True, False]


res = pd.DataFrame(columns=['dataset', 'obs_col', 'model', 'model_run',  'smote', 'test-set',
 'precision', 'recall', 'F1', 'accuracy', 'auc','avg_prec', 'feat.', 'label-imb-train', 
                            'label-imb-test', 'train-size', 'test-size', 'runtime(sec)'])


#model_runs = ["naive","remote","price",'lsms', 'infl', 'pricenorm',
#              'pricenorm+infl', 'pricenorm+remote', 'pricenorm+lsms',
#              "price+infl", "price+remote","price+lsms", 
#              "sat+lsms", "selected", "all"]

#if not already loaded in
data_coll=pd.read_csv("data_coll_allvars.csv")
data_coll = data_coll.drop(columns=['mzprice_yoy', 'region_central', 'mzprice_qtrchg'])

model_runs=['all']
outdir="Experiment 2 - SHAP"
if not os.path.exists(outdir):
    os.mkdir(outdir)

for obs_col in obs_cols:  
    selected_features = pd.DataFrame(columns=["wave", "dataset_num", "obs_col", "selectedvars"])
    if obs_col=="cat_cat1":
        smoteout=False
    else:
        smoteout=True
    for survey in surveys:
        for ds in range(3,4): #Just take the third dataset, as above; minor differences observed across all 5
            data_coll_wave=pd.read_csv(f"Subsamples/data_coll_samp{ds}_train_cat1.csv") #This has both cat1 and cat2
            data_coll_wave=data_coll_wave.drop(columns=['mzprice_yoy', 'region_central', 'mzprice_qtrchg'])
            data_coll_wave=data_coll_wave.loc[data_coll_wave['wave']<survey,:]
            for model_name in model_names:
                model = None
                if model_name == 'logistic-regression':
                    model = LogisticRegression(penalty='none')
                if model_name == 'lasso':
                    model= LogisticRegression(penalty='l1', solver='liblinear')
                if model_name == 'random-forest':
                    model = RandomForestClassifier()
                if model_name == 'tabnet':
                    model = TabNetClassifier(verbose=0,seed=42)
    
                for obs_col in tqdm(obs_cols):
                    for model_run in tqdm(model_runs):
                        if model_run=='naive':
                            allVars=naive_vars
                        elif model_run=="remote":
                            allVars= satVars+naive_vars
                        elif model_run=="price":
                            allVars= priceVars+naive_vars
                        elif model_run=="price+infl":
                            allVars=priceVars+inflVars+naive_vars
                        elif model_run=="price+lsms":
                            allVars=priceVars+lsmsVars+naive_vars
                        elif model_run=="lsms": 
                            allVars=lsmsVars+naive_vars
                        elif model_run=="sat+lsms":
                            allVars=satVars+lsmsVars+naive_vars
                        elif model_run=="infl+lsms":
                            allVars=inflVars+lsmsVars
                        elif model_run=="infl+remote":
                            allVars=inflVars+naive_vars+satVars
                        elif model_run=="infl+remote-naive":
                            allVars=inflVars+satVars
                        elif model_run=="selected":
                            allVars=selecVars
                        elif model_run=="infl":
                            allVars=inflVars+naive_vars
                        elif model_run=="all":
                            allVars=naive_vars + satVars + priceVars + lsmsVars+inflVars
    
                        if model_run!="selected":
                            resp_cols, _ = get_cols(data_coll_wave, allVars)
                        else:
                            resp_colls="allVars"
    
                        X_train_g=data_coll_wave.loc[:, resp_cols]
                        y_train_g=data_coll_wave.loc[:, obs_col]
                        
                        ts_df=data_coll.loc[data_coll['wave']==survey, :].reset_index(drop=True)
                        #resp_cols, _ = get_cols(data_coll, allVars)
                        X_test_g=ts_df.loc[:, resp_cols]
                        #y_test_g=ts_df.loc[:, obs_col]
                        
                        
                        if (smoteout==True) & (model_run!="naive"):
                            cat_vars = []
                            if 'region' in allVars:
                                cat_vars = cat_vars + ['region_south','region_north']
                            if 'rural' in allVars:
                                cat_vars = cat_vars + ['rural']
                            if 'qtr' in allVars:
                                cat_vars = cat_vars + ['qtr1','qtr2','qtr3']
                            if len(cat_vars) > 0:     
                                smote=SMOTENC(categorical_features=cat_vars, random_state=42)
                            else:
                                smote=SMOTE(random_state=42)
                            X_train_g, y_train_g=smote.fit_resample(X_train_g, y_train_g)
                            smote=True
                        else:
                            smote=False
                        
                        scaler = StandardScaler()
                        X_train_g = scaler.fit_transform(X_train_g)
                        X_test_g = scaler.transform(X_test_g)
                        start_time = time.time()
                        model.fit(X_train_g, y_train_g.values.ravel())
                        duration = time.time() - start_time
                        #y_pred = model.predict(X_test_g)
                        res= pd.concat([res, calculate_metrics_single(y_test_g, y_pred, duration, dataset, obs_col, model_name, model_run, 
                                                            smote, test_set, features_size, label_imb_train, label_imb_test, 
                                                            train_size, test_size)], axis=0)
                        shap_output_dir=os.path.join(outdir, f'SHAP_{obs_col}_{model_name}_{model_run}_w{survey}')
                        #modelout = model.steps[-1][1]
                        interpret_ml(X_train_g, X_test_g, resp_cols, model, shap_output_dir, model_run)
    res.to_csv(os.path.join(outdir, f"Exp2 Results {obs_col}.csv"))